In [1]:
# %load_ext autoreload
# %autoreload 2
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

In [2]:
gms8k = GSM8K()
turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=250)

trainset, devset = gms8k.train, gms8k.dev

dspy.settings.configure(lm=turbo)

100%|██████████| 1319/1319 [00:00<00:00, 40274.36it/s]


In [3]:
NUM_THREADS = 4
evaluate = Evaluate(devset=devset[:], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)

In [4]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)

In [5]:
RUN_FROM_SCRATCH = False

if RUN_FROM_SCRATCH:
    config = dict(max_bootstrapped_demos=8, max_labeled_demos=8, num_candidate_programs=10, num_threads=NUM_THREADS)
    teleprompter = BootstrapFewShotWithRandomSearch(metric=gsm8k_metric, **config)
    cot_bs = teleprompter.compile(CoT(), trainset=trainset, valset=devset)
    # cot_bs.save('turbo_8_8_10_gsm8k_200_300.json')
else:
    cot_bs = CoT()
    cot_bs.load('turbo_8_8_10_gsm8k_200_300.json')

In [6]:
evaluate(cot_bs, devset=devset[:])

Average Metric: 241 / 300  (80.3): 100%|██████████| 300/300 [00:00<00:00, 433.24it/s]


Average Metric: 241 / 300  (80.3%)


80.33

In [7]:
turbo.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Mark is baking bread. He has to let it rise for 120 minutes twice. He also needs to spend 10 minutes kneading it and 30 minutes baking it. How many minutes does it take Mark to finish making the bread?
Reasoning: Let's think step by step in order to find the total time it takes Mark to finish making the bread. We know that he has to let the bread rise for 120 minutes twice, which is 240 minutes. He also needs to spend 10 minutes kneading it and 30 minutes baking it. Therefore, the total time it takes Mark to finish making the bread is 240 + 10 + 30 = 280 minutes.
Answer: 280

---

Question: Ben has $2000 for his business operations costs. He orders goods from his supplier and writes them a cheque for $600. His debtor pays him $800 from the purchases they had